In [163]:
from pyhere import here
import os

os.chdir(here())

In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [165]:
in_filght_path = Path("data/raw/bats/All Yorkshire 2025 - In flight.xlsx")


bats_in_flight = pd.read_excel(in_filght_path,sheet_name="All Yorkshire In Flight")

bats_in_flight.head()

,ID,Source,Date,GridRef6Fig,Common name,Latin name,Activity type,Roosting?
0,2,NYBG,1996-06-10 00:00:00,SE653577,Noctule,Nyctalus noctula,Feeding,In Flight
1,9,NYBG,1997-04-09 00:00:00,SE295772,Noctule,Nyctalus noctula,Feeding,In Flight
2,11,NYBG,1997-06-09 00:00:00,SE464354,Noctule,Nyctalus noctula,Feeding,In Flight
3,14,NYBG,1997-06-24 00:00:00,SE396670,Noctule,Nyctalus noctula,Feeding,In Flight
4,37,NYBG,1997-10-17 00:00:00,SE295772,Noctule,Nyctalus noctula,Feeding,In Flight


In [166]:
roosting_path = Path("data/raw/bats/All Yorkshire 2025 - Roosting.xlsx")

bats_roosting = pd.read_excel(roosting_path,sheet_name="All Yorkshire 2025 - Roosting")

bats_roosting.head()


,ID,Source,Date,GridRef6Fig,Common name,Latin name,Activity type,Roosting?,Unnamed: 8
0,3,NYBG,1996-06-17,SE514602,Common pipistrelle,Pipistrellus pipistrellus,Maternity Roost,Roost,NaN
1,12,NYBG,1997-06-17,SD997629,Noctule,Nyctalus noctula,Maternity Roost,Roost,NaN
2,13,NYBG,1997-06-18,SE269624,Common pipistrelle,Pipistrellus pipistrellus,Maternity Roost,Roost,NaN
3,16,NYBG,1997-06-26,SE593526,Noctule,Nyctalus noctula,Maternity Roost,Roost,NaN
4,20,NYBG,1997-07-26,SE667794,Common pipistrelle,Pipistrellus pipistrellus,Summer Roost,Roost,NaN


# General Tidying

In [167]:
# strip trailing whitespace from column names
bats_in_flight.columns = bats_in_flight.columns.str.strip()
bats_roosting.columns = bats_roosting.columns.str.strip()

# convert to snake case
import re
def snake_case(s):
    # remove punctuation
    s = re.sub(r'[^\w\s]', '', s)
    # replace whitespace with underscore
    s = re.sub(r"\s+", "_", s).lower()
    return s

bats_in_flight.columns = [snake_case(col) for col in bats_in_flight.columns]
bats_roosting.columns = [snake_case(col) for col in bats_roosting.columns]

In [168]:
bats_in_flight["source_file"] = in_filght_path.name
bats_roosting["source_file"] = roosting_path.name

bats = pd.concat([bats_in_flight, bats_roosting])

bats.rename(columns={"gridref6fig": "grid_reference", "unnamed_8" : "info"}, inplace=True)

In [169]:
bats.head()

,id,source,date,grid_reference,common_name,latin_name,activity_type,roosting,source_file,info
0,2,NYBG,1996-06-10 00:00:00,SE653577,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN
1,9,NYBG,1997-04-09 00:00:00,SE295772,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN
2,11,NYBG,1997-06-09 00:00:00,SE464354,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN
3,14,NYBG,1997-06-24 00:00:00,SE396670,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN
4,37,NYBG,1997-10-17 00:00:00,SE295772,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN


## Overview of dataset

### Duplicate IDs

In [170]:
bats["unique_id"] = bats["source"] + bats["id"].astype(str)

duplicate_ids = bats["unique_id"].duplicated().sum()

print(f"Number of duplicate IDs: {duplicate_ids}")

Number of duplicate IDs: 0


### Species Names

In [171]:
bats["latin_name"] = bats["latin_name"].str.strip()

In [172]:
bats.latin_name.value_counts().sort_index()

Eptesicus serotinus              5
Myotis brandtii                 60
Myotis daubentonii             905
Myotis mystacinus              295
Myotis mystacinus/brandtii     151
Myotis mysticanus                1
Myotis nattereri               823
Myotis sp.                     311
Nyctalus leisleri              223
Nyctalus noctula               893
Nyctalus sp.                    39
Pipistrellus nathusii           48
Pipistrellus pipistrellus     7509
Pipistrellus pygmaeus         1744
Pipistrellus sp.              1518
Plecotus auritus              1811
Vespertilio murinus              1
Vespertilionidae              1714
Name: latin_name, dtype: int64

In [173]:
species_mapping = {
    "Myotis mysticanus": "Myotis mystacinus",
}

bats["latin_name"] = bats["latin_name"].replace(species_mapping)

bats.latin_name.value_counts().sort_index()

Eptesicus serotinus              5
Myotis brandtii                 60
Myotis daubentonii             905
Myotis mystacinus              296
Myotis mystacinus/brandtii     151
Myotis nattereri               823
Myotis sp.                     311
Nyctalus leisleri              223
Nyctalus noctula               893
Nyctalus sp.                    39
Pipistrellus nathusii           48
Pipistrellus pipistrellus     7509
Pipistrellus pygmaeus         1744
Pipistrellus sp.              1518
Plecotus auritus              1811
Vespertilio murinus              1
Vespertilionidae              1714
Name: latin_name, dtype: int64

In [ ]:
# Tag the latin names with the taxonomic level
taxonomic_levels = {
    "Pipistrellus pipistrellus": "species",
    "Pipistrellus pygmaeus": "species",
    "Nyctalus noctula": "species",
    "Vespertilionidae": "family",
    "Pipistrellus sp.": "genus",
    "Myotis daubentonii": "species",
    "Plecotus auritus": "species",
    "Myotis sp.": "genus",
    "Nyctalus leisleri": "species",
    "Myotis nattereri": "species",
    "Myotis mystacinus": "species",
    "Nyctalus sp.": "genus",
    "Myotis mystacinus/brandtii": "species_complex",
    "Pipistrellus nathusii": "species",
    "Myotis brandtii": "species",
    "Eptesicus serotinus": "species",
    "Vespertilio murinus": "species",
}

bats["taxonomic_level"] = bats["latin_name"].map(taxonomic_levels)

# raise a warning if any latin names are not in the mapping
missing_taxonomic_levels = bats["latin_name"][bats["taxonomic_level"].isnull()].unique()
if len(missing_taxonomic_levels) > 0:
    print(f"Warning: missing taxonomic levels for {missing_taxonomic_levels}")

We're only concerned about modelling the species, so we can drop any taxonomic levels which are not species.

In [175]:
bats[bats.taxonomic_level.isin(["species"])]

,id,source,date,grid_reference,common_name,latin_name,activity_type,roosting,source_file,info,unique_id,taxonomic_level
0,2,NYBG,1996-06-10 00:00:00,SE653577,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN,NYBG2,species
1,9,NYBG,1997-04-09 00:00:00,SE295772,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN,NYBG9,species
2,11,NYBG,1997-06-09 00:00:00,SE464354,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN,NYBG11,species
3,14,NYBG,1997-06-24 00:00:00,SE396670,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN,NYBG14,species
4,37,NYBG,1997-10-17 00:00:00,SE295772,Noctule,Nyctalus noctula,Feeding,In Flight,All Yorkshire 2025 - In flight.xlsx,NaN,NYBG37,species
...,...,...,...,...,...,...,...,...,...,...,...,...
11843,sybg7852,SYBG,2023-08-29 00:00:00,SK36908430,Common pipistrelle,Pipistrellus pipistrellus,NaN,Roost,All Yorkshire 2025 - Roosting.xlsx,NaN,SYBGsybg7852,species
11844,sybg7853,SYBG,2023-09-26 00:00:00,SK36908430,Common pipistrelle,Pipistrellus pipistrellus,NaN,Roost,All Yorkshire 2025 - Roosting.xlsx,NaN,SYBGsybg7853,species
11848,sybg7857,SYBG,2025-01-16 00:00:00,SK5346082911,Natterer's bat,Myotis nattereri,NaN,Roost,All Yorkshire 2025 - Roosting.xlsx,NaN,SYBGsybg7857,species
11849,sybg7858,SYBG,2025-01-16 00:00:00,SK5310083068,Soprano pipistrelle,Pipistrellus pygmaeus,NaN,Roost,All Yorkshire 2025 - Roosting.xlsx,NaN,SYBGsybg7858,species


# Grid References

In [176]:
# do some basic tidying up of removing whitespace and special characters

def clean_grid_reference(grid_reference):
    """
    Clean a grid reference by:
    - Stripping whitespace
    - Converting to uppercase
    - Removing anything that isn't a letter, number or space
    - replacing multiple spaces with a single space
    """
    grid_reference = grid_reference.str.strip()
    grid_reference = grid_reference.str.upper()
    grid_reference = grid_reference.str.replace(r"[^A-Z0-9 ]", "")
    grid_reference = grid_reference.str.replace(r"\s+", " ")

    return grid_reference

bats["grid_reference"] = clean_grid_reference(bats["grid_reference"])

/var/folders/w6/z6wv9w5s4r77c8m0lp63lm1w0000gp/T/ipykernel_50643/3780871441.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  grid_reference = grid_reference.str.replace(r"[^A-Z0-9 ]", "")
/var/folders/w6/z6wv9w5s4r77c8m0lp63lm1w0000gp/T/ipykernel_50643/3780871441.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  grid_reference = grid_reference.str.replace(r"\s+", " ")


In [186]:
import bng
import re
def check_grid_reference(grid_ref):
    if pd.isna(grid_ref) or not isinstance(grid_ref, str):
        return False
    pattern = r"^[A-Za-z]{2} ?\d{1,5} ?\d{1,5}$"
    return bool(re.match(pattern, grid_ref))

bats["grid_reference_valid"] = bats.grid_reference.apply(check_grid_reference)
bats[bats.grid_reference_valid == False][["source", "grid_reference"]].drop_duplicates()

,source,grid_reference
2021,EYBG,SE773116 57825
2280,EYBG,TA11021 128654
10557,SYBG,NaN


In [ ]:
grid_reference_mappings = {
    "S 4811699289": "SK4811699289", # Checked address in grid ref finder
    "SK401964toSK393980": "SK396970", # Taken estimated middle point
    "SK41894978784": "SK4189478784", # Checked post code in grid ref finder
    "SK401964TOSK393980": "SK396970", # Taken estimated middle point}
    "SE773116 57825" : "SE 77311 57825",
}
# Replace the invalid grid references
bats.grid_reference.replace(grid_reference_mappings, inplace=True)


In [188]:
bats["grid_reference_valid"] = bats.grid_reference.apply(check_grid_reference)
bats[bats.grid_reference_valid == False][["source_file", "date", "source", "id", "grid_reference", "latin_name"]]

,source_file,date,source,id,grid_reference,latin_name
2021,All Yorkshire 2025 - Roosting.xlsx,2022-06-09 00:00:00,EYBG,298,SE773116 57825,Plecotus auritus
2022,All Yorkshire 2025 - Roosting.xlsx,2022-06-06 00:00:00,EYBG,299,SE773116 57825,Pipistrellus pipistrellus
2023,All Yorkshire 2025 - Roosting.xlsx,2022-06-09 00:00:00,EYBG,300,SE773116 57825,Myotis nattereri
2024,All Yorkshire 2025 - Roosting.xlsx,2022-07-17 00:00:00,EYBG,301,SE773116 57825,Plecotus auritus
2025,All Yorkshire 2025 - Roosting.xlsx,2022-07-17 00:00:00,EYBG,302,SE773116 57825,Pipistrellus pipistrellus
2026,All Yorkshire 2025 - Roosting.xlsx,2022-07-17 00:00:00,EYBG,303,SE773116 57825,Myotis nattereri
2280,All Yorkshire 2025 - Roosting.xlsx,2018-08-20 00:00:00,EYBG,661,TA11021 128654,Pipistrellus pipistrellus
10557,All Yorkshire 2025 - Roosting.xlsx,2020-01-12 00:00:00,SYBG,sybg4515,NaN,Myotis nattereri
10558,All Yorkshire 2025 - Roosting.xlsx,2020-01-12 00:00:00,SYBG,sybg4516,NaN,Myotis daubentonii
10559,All Yorkshire 2025 - Roosting.xlsx,2020-01-12 00:00:00,SYBG,sybg4517,NaN,Plecotus auritus


In [189]:
bats.roosting.value_counts()

Roost        11851
In flight     5902
In Flight      530
Name: roosting, dtype: int64

# Save

In [178]:
from shapely.geometry import Polygon, Point
from datetime import datetime
output_schema = {
    "source": str,
    "source_id": int,
    "unique_id": str,
    "latin_name": str,
    "common_name": str,
    "activity_type": str,
    "date": datetime,
    "x" : float,
    "y" : float,
    "resolution" : float,
    "geometry": Polygon,
}